# 콘크리트 압축 강도 예측 모델 개발 프로젝트 개요

concrete_data.csv를 사용하여 콘크리트의 압축 강도를 예측하는 모델을 개발합니다.

스스로 데이터 불러오기에서부터 모델링 전략을 비롯하여 모델을 선택하고, 결과를 도출해봅니다.

미리 작성되어 있는 작업은 모두 구현해주시고 시간이 남으신다면 그 이외의 작업도 좋으니 스스로 생각하며 진행해봅니다.

코드 셀은 얼마든지 새로 추가하고 삭제하시며 사용하셔도 좋습니다.

**수업 시간**   
00분\~50분 | 프로젝트 수행   
50분\~00분 | 쉬는시간   

## 제출 방법 및 평가 기준
**제출 방법**
- 구글 폼에 답안을 작성한 ipynb 파일을 첨부해 주세요!
- 답안 제출 기한은 08.23(토) 23시 59분까지 입니다!
- 이름_과제5.ipynb 으로 제출 해주시고 파일 제출 전 셀들의 실행 결과를 출력 (print) 해주세요.
- 성능이 가장 좋은 모델의 성능을 마지막 셀에서 출력해주세요.
- 작성된 모든 코드는 위에서부터 아래 방향으로 셀 단위로 실행되도록 작성해주세요.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

### 데이터셋 개요
- 이 데이터셋은 콘크리트의 다양한 구성 요소와 그에 따른 압축 강도를 포함하고 있는 데이터셋입니다.   

- 총 1030개의 샘플이 있으며, 각 샘플은 콘크리트를 구성하는 재료의 양과 콘크리트가 경화된 후의 압축 강도를 나타냅니다.   

- 이 데이터셋은 콘크리트의 압축 강도를 예측하기 위한 머신러닝 모델을 개발하는 데 사용됩니다.   

- 우리가 예측하려는 것은 콘크리트의 압축 강도(concrete_compressive_strength)입니다.   




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = "/content/drive/MyDrive/CDS/week5/project/concrete_data.csv"
df = pd.read_csv(file_path)
df

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


**df의 각 컬럼 설명**   

- **cement:** 콘크리트를 만들 때 사용되는 시멘트의 양입니다. 시멘트는 물과 결합하여 콘크리트를 단단하게 만드는 중요한 재료입니다. (단위: kg)   

- **blast_furnace_slag:** 철을 만들 때 나오는 부산물인 고로 슬래그의 양입니다. 이 재료는 콘크리트의 강도를 높이고 내구성을 향상시키는 데 사용됩니다. (단위: kg)   

- **fly_ash:** 석탄을 태울 때 발생하는 미세한 재로, 플라이 애쉬의 양입니다. 플라이 애쉬는 콘크리트의 작업성을 개선하고 장기적인 강도를 증가시킵니다. (단위: kg)   

- **water:** 콘크리트를 혼합할 때 사용되는 물의 양입니다. 물은 시멘트와 반응하여 콘크리트를 굳게 만듭니다. (단위: kg)   

- **superplasticizer:** 콘크리트의 유동성을 높여주는 화학 첨가제의 양입니다. 이 첨가제는 물의 사용량을 줄이면서도 콘크리트를 쉽게 다룰 수 있게 합니다. (단위: kg)   

- **coarse_aggregate:** 콘크리트에 포함된 큰 자갈이나 돌의 양입니다. 굵은 골재는 콘크리트의 구조적 강도를 제공하는 역할을 합니다. (단위: kg)   

- **fine_aggregate:** 콘크리트에 포함된 모래와 같은 작은 입자의 양입니다. 잔골재는 콘크리트의 표면을 매끄럽게 하고 강도를 보강합니다. (단위: kg)   

- **age:** 콘크리트가 굳어진 후 경과된 시간입니다. 일반적으로 일 단위로 측정되며, 콘크리트의 강도는 시간이 지남에 따라 증가합니다. (단위: 일)   

- **concrete_compressive_strength:** 콘크리트가 압축되는 힘을 견디는 능력입니다. 이 값은 콘크리트의 품질과 내구성을 평가하는 중요한 지표입니다. (단위: MPa)   

   
   


#### 1. 모델링을 위한 데이터 전처리를 진행합니다.   
- X, y 분리   

- train, test 분리   

- 스케일링   

- 5-fold 적용   

    - kfold의 random_state는 42로 고정, shuffle은 True로 설정   
    


X, y 분리

In [ ]:
X, y = df.drop(columns=['concrete_compressive_strength']), df['concrete_compressive_strength']

Train Test 분리

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42 , shuffle=True)

K-Fold

In [ ]:
def create_kfold_splits(X_train, y_train, n_splits=5, random_state=42):
    """KFold를 사용하여 데이터 분할을 생성하는 함수"""
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    fold_results = []
    for train_index, val_index in kf.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
        fold_results.append((X_train_fold, X_val_fold, y_train_fold, y_val_fold))
    return fold_results

# 함수 호출
fold_results = create_kfold_splits(X_train, y_train)

정규화 적용

In [ ]:
# 정규화 적용
scaler = StandardScaler()

# fold_results_no_outliers의 각 fold에 대해 정규화 적용
fold_results_scaled = []
for X_train_fold, X_val_fold, y_train_fold, y_val_fold in fold_results:
    X_train_fold_scaled = scaler.fit_transform(X_train_fold)
    X_val_fold_scaled = scaler.transform(X_val_fold)
    fold_results_scaled.append((X_train_fold_scaled, X_val_fold_scaled, y_train_fold, y_val_fold))


#### 2. 모델 구현 및 성능을 테스트합니다.

- Linear Regression, Decision Tree, Random Forest, Gradient Boosted Tree 4개의 모형을 구현해봅시다.

- 최대한의 성능을 스스로 구현해보고 성능을 테스트해봅니다.   

- 모델의 random_state는 42로 고정   

- 4개의 모형을 구현하여, 테스트 결과를 데이터프레임 형태로 출력합니다.

- 평가지표는 MAE, MSE, R2 를 사용합니다

In [ ]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    # 모델 학습
    model.fit(X_train, y_train)

    # 테스트 데이터로 예측
    y_pred = model.predict(X_test)

    # 예측 성능 평가 (평균 절대 오차, R2 스코어, 평균 제곱 오차)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    return mse, mae, r2

In [ ]:
def add_evaluation_results(test_name, mse, mae, r2, results=None):
    new_results = pd.DataFrame([
        [test_name, mse, mae, r2]
    ], columns=['test_name', 'Mean Squared Error', 'Mean Absolute Error', 'R2 Score'])

    if results is not None:
      new_results = pd.concat([results, new_results], ignore_index=True)

    return new_results

In [ ]:
models = [

    [LinearRegression(), 'LinearRegression'],
    [DecisionTreeRegressor(), 'DecisionTreeRegressor'],
    [RandomForestRegressor(random_state=42, n_estimators=200), 'random_forest'],
    [GradientBoostingRegressor(random_state=42, n_estimators=1000), 'GradientBoostingRegressor'],
]

# 5-fold evaluation
results = None
for X_train_fold, X_val_fold, y_train_fold, y_val_fold in fold_results_scaled:
  # 각 모델에 대해 평가 및 결과 기록

  for model, name in models:
      mse, mae, r2 = evaluate_model(model, X_train, X_test, y_train, y_test)
      results = add_evaluation_results(name, mse, mae, r2, results=results)


display(results.groupby('test_name').mean())

,Mean Squared Error,Mean Absolute Error,R2 Score
test_name,,,
DecisionTreeRegressor,44.056170,4.382714,0.829026
GradientBoostingRegressor,18.713543,2.822110,0.927376
LinearRegression,95.970940,7.745559,0.627553
random_forest,30.554012,3.781352,0.881425


- 최종적으로 모형을 하나 선택하고, 왜 이 모형을 선택하였는지 해석해봅시다.

해석 :